# Interrupción Legal del Embarazo

## Parameters

In [1]:
BASE_DIR = '/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data' # Mac
# BASE_DIR = r'D:\Users\efras\Documents\EF\Contests\Datathon_CDMX\data' # Windows
RESOURCE_ID = '932b56bf-c5ec-4815-9814-370d58754002' ##### API SÓLO TRAE INFO 2016
FILE_NAME = 'interrupcion_embarazo'

## Import

In [2]:
from mariachis.models import InterrupcionEmbarazo
ile = InterrupcionEmbarazo(BASE_DIR, FILE_NAME)
print(ile)

Directorio: 	/Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


In [3]:
##### API SÓLO TRAE INFO 2016
ile.full_import(api=True, api_export=False, resource_id=RESOURCE_ID)['ano'].value_counts(1)

Archivo importado desde: https://datos.cdmx.gob.mx/api/3/action/datastore_search?resource_id=932b56bf-c5ec-4815-9814-370d58754002&distinct=true&limit=32000
Con 1000 renglones y 48 columnas


2016    1.0
Name: ano, dtype: float64

In [4]:
df = ile.full_import(api=False)
df.sample()

Archivo con nombre interrupcion_embarazo.csv fue encontrado en /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data
Con 79403 renglones y 47 columnas


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,tanalgesico,cconsejo,panticoncep,fecha_cierre,resultado_ile,medicamento,clues_2,clues_hospital,procile_simplificada,procile_completa
58054,2018.0,FEBRERO,2018-02-12,NaN,UNION LIBRE,24.0,SEGURO POPULAR,PRIMARIA,TRABAJADORA DEL HOGAR NO REMUNERADA,NINGUNA,...,NaN,SI,IMPLANTE SUBDERMICO,NaN,COMPLETA,NaN,DFSSA004270,DFSSA004270,MEDICAMENTO,MISOPROSTOL + MIFEPRISTONA


## Dict

In [5]:
# Diccionario para aplicar limpieza por tipo de variable
full_dict = {}

# Variables que o son 0 o > 0
full_dict['vars_numbin'] = ['naborto', 'npartos', 'ncesarea', 'nile']

# Numéricas que serán partidas en rangos
full_dict['vars_num'] = {
    'edad': [9,11,15,18,21,25,29,35],
    'menarca': [7,11,12,13,14,15],
    'fsexual': [7,11,15,18,21],
    'sememb': [3,6,9,12,20],
    'nhijos': [0,1,2],
    'gesta': [0,1,2,3,4],
    'c_num': [0,1,2],
    'p_semgest': [0,5,6,7,8,9],
    'p_diasgesta': [0,1,2,3,4,5]
}

# Obtener la primer palabra
full_dict['vars_first_word'] = ['anticonceptivo',  'panticoncep']

# Binarias de sí o no
full_dict['vars_yes_no'] = ['autoref', 'consejeria', 'p_consent', 's_complica', 'c_dolor', 'tanalgesico', 'cconsejo', 'resultado_ile']

# Sólo cambiar nulos por "DESCONOCIDO"
full_dict['vars_just_fill_na'] = ['entidad', 'motiles', 'desc_servicio', 'anticonceptivo', 'panticoncep']

# Agrupar categorías
full_dict['vars_cat'] = {
    'edocivil_descripcion':{
        'SOLTERA':'SOLTERX', 
        'UNION LIBRE':'UNION_LIBRE', 
        'CASADA':'CASADX',
        'DIVORCIADA':'SEPARADX',
        'SEPARADA':'SEPARADX',
        'VIUDA':'SEPARADX',
    },
    'ocupacion':{
        'EMPLEADA':'EMPLEADX',
        'ESTUDIANTE':'ESTUDIANTE',
        'TRABAJADORA DEL HOGAR NO REMUNERADA':'TRAB_HOGAR_NO_REMUNERADX',
        'DESEMPLEADA':'DESEMPLEADX',
    },
    'desc_derechohab':{
        'NINGUNO':'NINGUNO',
        'IMSS':'ALGUNO',
        'SEGURO POPULAR':'ALGUNO',
        'OTRA':'ALGUNO',
        'ISSSTE':'ALGUNO',
    },
    'nivel_edu':{
        'LICENCIATURA':'LICENCIATURA',
        'MAESTRIA':'POSGRADO',
        'DOCTORADO':'POSGRADO',
        'PREPARATORIA':'PREPARATORIA',
        'SECUNDARIA':'SECUNDARIA',
        'PRIMARIA':'PRIMARIA',
        'SIN ACCESO A LA EDUCACION FORMAL':'SIN_ACCESO',
    },
    'parentesco':{
        'PAREJA':'PAREJA',
        'EX-PAREJA':'PAREJA',
        'FAMILIAR MUJER':'FAMILIAR',
        'FAMILIAR HOMBRE':'FAMILIAR',
        'FAMILIAR SIN ESPECIFICAR':'FAMILIAR',
        'TUTOR O RESPONSABLE LEGAL':'FAMILIAR',
        'PERSONA CERCANA':'AMIGX',
        'PERSONA CERCANA MUJER':'AMIGX',
        'PERSONA CERCANA HOMBRE':'AMIGX',
        'AMIGA':'AMIGX',
        'AMIGO':'AMIGX',
        'CONOCIDA':'OTRX',
        'CONOCIDO':'OTRX',
        'RELACION LABORAL':'OTRX',
        'OTRA':'OTRX',
    },
    'religion':{
        'NINGUNA':'NINGUNA',
        'CATOLICA':'SI',
        'CRISTIANA':'SI',
        'OTRA':'SI',
        'MORMONA':'SI',
        'TESTIGA DE JEHOVA':'SI',
        'BUDISTA':'SI',
        'JUDIA':'SI',
        'ANGLICANA':'SI',
        'MUSULMANA':'SI',
    },
}

### Auxiliar para rangos de variables numéricas

In [8]:
from pandas import qcut
from IPython.display import display

aux_dict = {}
for col in full_dict['vars_num']:
    new_col = df[col].map(ile.clean_number).astype(float)
    orig_bins = qcut(new_col, q=6, retbins=True, duplicates='drop')[-1]
    aux_dict[col] = list(orig_bins[:-1])
display(aux_dict)

{'edad': [11.0, 20.0, 22.0, 25.0, 28.0, 32.0],
 'menarca': [0.0, 11.0, 12.0, 13.0, 14.0],
 'fsexual': [0.0, 15.0, 16.0, 17.0, 18.0, 19.0],
 'sememb': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'nhijos': [0.0, 1.0, 2.0],
 'gesta': [0.0, 1.0, 2.0, 3.0, 4.0],
 'c_num': [0.0, 1.0, 2.0],
 'p_semgest': [0.0, 5.0, 6.0, 7.0, 8.0, 9.0],
 'p_diasgesta': [0.0, 1.0, 2.0, 3.0, 4.0, 5.0]}

## Transform

In [9]:
# Quitar comentarios para ejecutar proceso. Nota: Demoró 13min con 77K registros
X = ile.wrangling_aborto(df, full_dict, export_result=True)
X.sample(4)

Archivo: interrupcion_embarazo_cluster.csv fue exportado exitosamente en: /Users/efraflores/Desktop/EF/Contests/Datathon_CDMX/data


,año,mes,fingreso,autoref,edocivil_descripcion,edad,desc_derechohab,nivel_edu,ocupacion,religion,...,rango_edad,rango_menarca,rango_fsexual,rango_sememb,rango_nhijos,rango_gesta,rango_c_num,rango_p_semgest,rango_p_diasgesta,cluster
41977,2016.0,MARZO,2016-03-15 00:00:00,SI,UNION_LIBRE,26.0,DESCONOCIDO,PRIMARIA,TRAB_HOGAR_NO_REMUNERADX,SI,...,26 a 29,14,16 a 18,04 a 06,> 02,04,> 02,06,02,E
66358,2016.0,ABRIL,2016-04-29 00:00:00,DESCONOCIDO,SOLTERX,32.0,NINGUNO,PREPARATORIA,EMPLEADX,SI,...,30 a 35,12,19 a 21,07 a 09,01,02,01,07,03,C
35141,2019.0,ENERO,2019-01-30 00:00:00,DESCONOCIDO,SEPARADX,29.0,NINGUNO,SECUNDARIA,EMPLEADX,SI,...,26 a 29,15,16 a 18,04 a 06,01,03,01,01 a 05,DESCONOCIDO,E
53876,2017.0,MARZO,2017-03-24 00:00:00,DESCONOCIDO,SOLTERX,20.0,ALGUNO,PREPARATORIA,ESTUDIANTE,DESCONOCIDO,...,19 a 21,08 a 11,19 a 21,07 a 09,00,02,DESCONOCIDO,01 a 05,> 05,C


## Checkpoint

In [ ]:
from pandas import read_csv
from pathlib import Path

# Si no se desea re-clusterizar, entonces mantener comentarios de la sección pasada e importar el resultado desde esta celda
vars_cat = list(set(
            full_dict['vars_first_word']+
            list(full_dict['vars_cat'].keys())+
            full_dict['vars_yes_no']+
            full_dict['vars_just_fill_na']
        ))
cluster_cols = vars_cat+[f'rango_{col}' for col in full_dict['vars_num'].keys()]+full_dict['vars_numbin']
add_cols = ['año','mes','alc_o_municipio']

df = read_csv(Path(BASE_DIR).joinpath(FILE_NAME+'_cluster.csv'), low_memory=False)
df.sample()

## Perfilamiento

In [ ]:
ile.profiles(df[cluster_cols+add_cols+['cluster']])